In [17]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean


In [18]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())

digenkeys = list(digen.keys())

#print(digenkeys)
#print(digen['digen8_4426'])
digennoise = {}

In [19]:
for i in digenkeys: 
    data = pd.DataFrame(digen[i]['X'])
    yval = pd.DataFrame(digen[i]['y'])
    current_noise = digen[i]['noise']
    yval.columns = ['y']
    yval.rename(columns = {'0':'y'}, inplace = True)
    out = pd.concat([Test, yval], axis=1)
    out.index.name = 'InstanceId'
    digennoise[i] = current_noise
    if current_noise == 0.0: 
        outstring = './data/DigenStudy/No_Noise/' + i + '.csv'
        out.to_csv(outstring)
    else:
        outstring = './data/DigenStudy/Noisy/' + i + '.csv'
        out.to_csv(outstring)



In [20]:
#Noisy checks:



type: 'float' object is not subscriptable